# Rcpp入門 その3

「オリジナルのR」、「RcppArmadillo」、「RcppEigen」で処理速度の比較をしてみたいと思います。

今回はソートで比較してみます。

「オリジナルのR」は`order()`、「Armadillo」は`sort_index()`、「Eigen」は`std::sort()`を使います。

※ バージョンによっては正常に動作しない場合があります。私が試した環境は、

* R version 3.4.1
* Rcpp 0.12.12
* RcppArmadillo 0.7.960.1.2
* RcppEigen 0.3.3.3.0

##  Armadilloは、sort_index()で実装

In [1]:
# パッケージの読込み
library(Rcpp)
library(RcppArmadillo)

# C+11にてコンパイルします
Sys.setenv("PKG_CXXFLAGS"="-std=c++11")

In [2]:
# C++のコード
code <- '
arma::uvec IndexSortArma(arma::vec src)
{
    // ソート
    arma::uvec index = sort_index(src);

    return index + 1; //Rのインデックスは1スタートなので、1追加
} 
'

# コンパイルとリンクを行います（少し時間がかかります）
cppIndexSortArma <- cppFunction(code, depends="RcppArmadillo")

## Eigenは、std::sort()で実装

Eigenにはソート関数が無いようなので、std::sort()を使います。

[std::vectorをインデックスをそのままにラムダ式でsortする](http://qiita.com/Nasupl_r/items/35a903a4a69932c26c83)を参考にしました。

In [3]:
library(Rcpp)
library(RcppEigen)

# C+11にてコンパイルします
Sys.setenv("PKG_CXXFLAGS"="-std=c++11")


Attaching package: 'RcppEigen'

The following objects are masked from 'package:RcppArmadillo':

    fastLm, fastLmPure



In [4]:
# C++のコード
code <- '
Eigen::VectorXi IndexSortEigen(Eigen::VectorXd src)
{
    // 出力するインデックス
    int num = src.size();
    Eigen::VectorXi index(num);
    std::iota(index.data(), index.data() + num, 0);

    // ソート
    std::sort(
        index.data(),
        index.data() + num,
        [&src](double x, double y) -> bool { return src[x] < src[y]; }
    );

    return index.array() + 1; //Rのインデックスは1スタートなので、1追加
}
'

# コンパイルとリンクを行います（少し時間がかかります）
cppIndexSortEigen <- cppFunction(code, depends="RcppEigen", includes=c('#include <RcppEigen.h>'))

## 結果

元データを作成します。

In [5]:
src <- rnorm(10000000)

**「オリジナルのR」の`order()`**

In [6]:
t <- proc.time()

dst_r <- order(src)

print(proc.time() - t)

   user  system elapsed 
   0.72    0.03    0.75 


**「Armadillo」の`sort_index()`**

In [7]:
t <- proc.time()

dst_arma <- cppIndexSortArma(src)

print(proc.time() - t)

   user  system elapsed 
   1.02    0.08    1.09 


**「Eigen」については、`std::sort()`**

In [8]:
t <- proc.time()

dst_eigen <- cppIndexSortEigen(src)

print(proc.time() - t)

   user  system elapsed 
   2.29    0.03    2.32 


**結果のチェック**

In [9]:
print("index_sort()の結果チェック")
all(dst_r == dst_arma)
print("std::sort()の結果チェック")
all(dst_r == dst_eigen)

[1] "index_sort()の結果チェック"


[1] TRUE

[1] "std::sort()の結果チェック"


[1] TRUE

今回の結果としては、以下の順位になりました。

1. 「オリジナルのR」の`order()` 
1. 「Armadillo」の`sort_index()` 
1. 「Eigen」については、`std::sort()`

「Armadillo」や「Eigen」については、この結果でライブラリの性能についての良し悪しは言えないと思います。「Eigen」にいたっては、`std::sort()`を使っていますし。

ただし、以下のことは言えると思っています。

* 「オリジナルのR」の標準の関数は結構速い
* 「Armadillo」や「Eigen」を使いさえすれば高速化できるというわけではない

----
[ホームへ](https://waku-take-a.github.io/index.html)  
[↑Rの記事Topへ](https://waku-take-a.github.io/R.html)  

※  
上記のipynbファイル等は、[こちらに](https://github.com/WAKU-TAKE-A/RTips)あります。  
解凍後、`The jupyter notebook`にアップロードすれば、ローカルな環境で実行することも、編集することもできます。

<(_ _)>

★★★responsive_ja★★★